In [1]:
import os.path
import pandas as pd
%cd ~/23.03_DHODH

/Users/jjaenes/23.03_DHODH


In [28]:
fp_ = 'adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.smi'
pd.read_csv('resources/DHODH_ChEMBL/DHODH_ChEMBL.tsv', sep='\t')[['canonical_smiles', 'pchembl_value']].to_csv(fp_, index=False, header=False, sep='\t')
!head -n 5 adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.smi

Brc1ccc(-c2csc(N/N=C/c3ccccc3)n2)cc1	5.48
C/C(=C\C[C@@]1(C)[C@H](C)CCC(=O)[C@@H]1C)[C@@H](O)Cc1c(O)c(Cl)c(C)c(C=O)c1O	5.33
C/C(=C\Cc1c(O)c(Cl)c(C)c(C(=O)O)c1O)CC/C=C(\C)CC[C@H](O)C(C)(C)O	4.7
C/C(=C\Cc1c(O)c(Cl)c(C)c(C=O)c1O)CC/C(C)=C(\C)C(C)CCCO	4.5
C/C(=C\Cc1c(O)c(Cl)c(C)c(C=O)c1O)CC/C=C(\C)CCC(=O)C(C)(C)O	5.03


In [29]:
!conda run -p envs/qsar-tools software/qsar-tools/createsmartsdescriptors.py --subgraph -c 3 adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.smi -o adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL_smarts.txt

In [30]:
!conda run -p envs/qsar-tools software/qsar-tools/outputfingerprints.py --smarts --smartsfile adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL_smarts.txt adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.smi -o adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.fp.gz

Could not import pybel: FP2 fingerprints not supported



In [38]:
!conda run -p envs/qsar-tools software/qsar-tools/trainlinearmodel.py --elastic adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.fp.gz -o adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.model

Elastic alpha = 0.013441391372939432  l1_ratio = 0.1
Full Regression: R^2=0.8425, RMS=0.5127, NullRMS=1.2760
Nonzeros: 1667 (19.39%)
3-fold CV: R^2=0.6470, RMS=0.7590, NullRMS=1.2734 (stds 0.0287, 0.0181, 0.0607)
Gap: R^2=0.1955, RMS=-0.2463, NullRMS=0.0026

/Users/jjaenes/23.03_DHODH/envs/qsar-tools/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.612e-01, tolerance: 1.175e-01
  model = cd_fast.enet_coordinate_descent(
/Users/jjaenes/23.03_DHODH/envs/qsar-tools/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.794e-01, tolerance: 1.175e-01
  model = cd_fast.enet

In [39]:
!conda run -p envs/qsar-tools software/qsar-tools/outputfingerprints.py --smarts --smartsfile adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL_smarts.txt resources/DHODH_assay/DHODH_Inhibitors_LS.smi -o adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_LS.fp.gz
!conda run -p envs/qsar-tools software/qsar-tools/outputfingerprints.py --smarts --smartsfile adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL_smarts.txt resources/DHODH_assay/DHODH_inhibitors_pubchem.smi -o adhoc/A_QSAR_pChEMBL/DHODH_inhibitors_pubchem.fp.gz

Could not import pybel: FP2 fingerprints not supported

Could not import pybel: FP2 fingerprints not supported



In [40]:
!conda run -p envs/qsar-tools software/qsar-tools/applylinearmodel.py adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.model adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_LS.fp.gz > adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_LS.applylinearmodel.txt
!conda run -p envs/qsar-tools software/qsar-tools/applylinearmodel.py adhoc/A_QSAR_pChEMBL/DHODH_pChEMBL.model adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_pubchem.fp.gz > adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_pubchem.applylinearmodel.txt

In [41]:
pd.read_csv('adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_LS.applylinearmodel.txt', names=['smiles', 'pChEMBL_predicted'], delim_whitespace=True).merge(
    right=pd.read_csv('resources/DHODH_assay/DHODH_Inhibitors_LS.smi', names=['smiles', 'compound'], delim_whitespace=True),
    on='smiles'
)[['compound', 'pChEMBL_predicted']].sort_values('pChEMBL_predicted', ascending=False).reset_index(drop=True)

,compound,pChEMBL_predicted
0,Teriflunomide,6.210046
1,Felodipine,5.726989
2,Leflunomide,5.690177
3,Flunarizine_dihydrochloride,5.456146
4,Tegaserod_maleate,5.293396
5,Tiratricol,5.174487
6,Amethopterin,4.914581
7,Dicumarol,4.816853
8,Cisatracurium_besylate,4.714666
9,Mesoridazine_besylate,4.664790


In [43]:
pd.read_csv('adhoc/A_QSAR_pChEMBL/DHODH_Inhibitors_pubchem.applylinearmodel.txt', names=['smiles', 'pChEMBL_predicted'], delim_whitespace=True).merge(
    right=pd.read_csv('resources/DHODH_assay/DHODH_inhibitors_pubchem.smi', names=['smiles', 'compound'], delim_whitespace=True),
    on='smiles'
)[['compound', 'pChEMBL_predicted']].sort_values('pChEMBL_predicted', ascending=False).reset_index(drop=True)

,compound,pChEMBL_predicted
0,Teriflunomide,6.210046
1,Felodipine,5.726989
2,Leflunomide,5.690177
3,Tegaserod_maleate,5.532792
4,Flunarizine_dihydrochloride,5.456146
5,Tiratricol,5.174487
6,Amethopterin,4.914581
7,Dicumarol,4.816853
8,Cisatracurium_besylate,4.761155
9,Mesoridazine_besylate,4.669875
